In [10]:
import pandas as pd
import sqlite3

In [11]:
df = pd.read_csv("./Ticker.csv")

In [12]:
df.head(4)

,Date,Close,Ticker
0,2021-08-20,88.940002,ORCL
1,2021-08-23,89.120003,ORCL
2,2021-08-24,88.550003,ORCL
3,2021-08-25,88.620003,ORCL


## max close price for each ticker

Calculate the maximum close price for each ticker by using the MAX function in conjunction with a window function

Window function is initiated with the OVER keyword

Specify that the ticker column to partition the data


It will return a table of the same length as the original, but with a new column called MaxPrice that contains the maximum close price for ticker during time period

In [13]:
import sqlite3

# create connection to in memory sqlite db
conn = sqlite3.connect(":memory:")

# save prices dataframe to sqlite db
df.to_sql(name="TickerPriceTable", con=conn, index=False)


query1 = """
    
    SELECT
        date(Date) as Date
        , Ticker
        , Close as ClosingPrice
        , MAX(Close) OVER(PARTITION BY Ticker) as MaxPrice
    FROM
        TickerPriceTable
        
"""

# use pandas read_sql to execute the query and return a dataframe
result1_df = pd.read_sql(query1, con=conn)

In [14]:
result1_df.head(10)

,Date,Ticker,ClosingPrice,MaxPrice
0,2021-08-20,MAZN,159.997498,184.802994
1,2021-08-23,MAZN,163.293503,184.802994
2,2021-08-24,MAZN,165.289001,184.802994
3,2021-08-25,MAZN,164.959000,184.802994
4,2021-08-26,MAZN,165.800003,184.802994
5,2021-08-27,MAZN,167.481506,184.802994
6,2021-08-30,MAZN,171.078506,184.802994
7,2021-08-31,MAZN,173.539505,184.802994
8,2021-09-01,MAZN,173.949997,184.802994
9,2021-09-02,MAZN,173.156006,184.802994


## 30 day closing price moving average for each Ticker 

For moving average computation - the order of values within the group are important (It should be in chronological order), 
therefore need to order the values within the group by the date

To define the rolling window, specify that the average should be calculated using the preceding *n* rows & the current row



In [15]:
query2 = """

SELECT
    date(Date) AS Date
    , Ticker
    , Close
    , AVG(Close) OVER(
        PARTITION BY Ticker
        ORDER BY date(Date)
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    )
     AS MovingAvg_30_Days
FROM
    TickerPriceTable
    
"""

In [16]:
# use pandas read_sql to execute the query and return a dataframe
result2_df = pd.read_sql(query2, con=conn)

In [17]:
result2_df.head(10)

,Date,Ticker,Close,MovingAvg_30_Days
0,2021-08-20,MAZN,159.997498,159.997498
1,2021-08-23,MAZN,163.293503,161.645500
2,2021-08-24,MAZN,165.289001,162.860001
3,2021-08-25,MAZN,164.959000,163.384750
4,2021-08-26,MAZN,165.800003,163.867801
5,2021-08-27,MAZN,167.481506,164.470085
6,2021-08-30,MAZN,171.078506,165.414145
7,2021-08-31,MAZN,173.539505,166.429815
8,2021-09-01,MAZN,173.949997,167.265391
9,2021-09-02,MAZN,173.156006,167.854452


## Get Previous day's Close Price for each Ticker 


In [18]:
query3 = """

SELECT
    date(Date) AS Date
    , Ticker
    , Close
    , LAG(Close, 1) OVER(
        PARTITION BY Ticker
        ORDER BY date(Date)
    ) AS PreviousClose
FROM
    TickerPriceTable
    
"""
 

In [19]:
# use pandas read_sql to execute the query and return a dataframe
result3_df = pd.read_sql(query3, con=conn)

In [20]:
result3_df.head(10)

,Date,Ticker,Close,PreviousClose
0,2021-08-20,MAZN,159.997498,NaN
1,2021-08-23,MAZN,163.293503,159.997498
2,2021-08-24,MAZN,165.289001,163.293503
3,2021-08-25,MAZN,164.959000,165.289001
4,2021-08-26,MAZN,165.800003,164.959000
5,2021-08-27,MAZN,167.481506,165.800003
6,2021-08-30,MAZN,171.078506,167.481506
7,2021-08-31,MAZN,173.539505,171.078506
8,2021-09-01,MAZN,173.949997,173.539505
9,2021-09-02,MAZN,173.156006,173.949997


## Daily Percentage Return 

In [21]:
query4 = """

WITH PreviousCloseTickerPrices AS (
SELECT
     date(Date) AS Date,
     Ticker,
     Close,
     LAG(Close, 1) OVER(PARTITION BY Ticker ORDER BY date(Date)) AS PreviousClose
FROM
    TickerPriceTable
)

SELECT
    Date,
    Ticker,
    Close,
    Close/PreviousClose - 1 AS DailyReturn
FROM PreviousCloseTickerPrices

"""

In [22]:
# use pandas read_sql to execute the query and return a dataframe
result4_df = pd.read_sql(query4, con=conn)

In [23]:
result4_df.head(10)

,Date,Ticker,Close,DailyReturn
0,2021-08-20,MAZN,159.997498,NaN
1,2021-08-23,MAZN,163.293503,0.020600
2,2021-08-24,MAZN,165.289001,0.012220
3,2021-08-25,MAZN,164.959000,-0.001997
4,2021-08-26,MAZN,165.800003,0.005098
5,2021-08-27,MAZN,167.481506,0.010142
6,2021-08-30,MAZN,171.078506,0.021477
7,2021-08-31,MAZN,173.539505,0.014385
8,2021-09-01,MAZN,173.949997,0.002365
9,2021-09-02,MAZN,173.156006,-0.004564
